In [2]:
import math, random, json
import numpy as np
from PIL import Image, ImageDraw, ImageFont

uid_counter = 0
def get_new_uid():
    global uid_counter
    uid_counter += 1
    return uid_counter

best_counter = 0
def save_new_best(path, score):
    global best_counter
    best_counter += 1
    score = round(score)
    with open(f"exports/{best_counter:09}-{score:05}.txt", "w") as file:
        json.dump(path, file)

class Station:

    def __init__(self, position=None, distance_map=None):
        self.position: None | np.ndarray = position
        self.distance_map: dict["Station": float] = {} if distance_map is None else distance_map
        self.uid = get_new_uid()

    def __hash__(self):
        return hash(self.uid)

    def __eq__(self, other):
        if isinstance(other, Station):
            return self.uid == other.uid
        raise NotImplementedError()

    def __ne__(self, other):
        return not self.__eq__(other)

    def __lt__(self, other):
        if isinstance(other, Station):
            return self.uid < other.uid
        raise NotImplementedError()

    def __gt__(self, other):
        return not self.__lt__(other) and not self.__eq__(other)

    def __repr__(self):
        if self.position is None:
            return f"{self.__class__.__name__}(uid={self.uid})"
        else:
            return f"{self.__class__.__name__}(position={self.position!r})"

class TramNetwork:

    def __init__(self, stations=None):
        self.stations: list[Station] = stations if stations is not None else []

    @classmethod
    def generate_random(cls, num_stations=10) -> "TramNetwork":
        stations: list[Station] = []
        for _ in range(num_stations):
            position = np.array([
                random.random(),
                random.random()
            ])
            stations.append(Station(position))

        for station in stations:
            station.distance_map = {}
            for other_station in stations:
                if station == other_station:
                    distance = 0
                else:
                    distance = np.linalg.norm(station.position - other_station.position)
                station.distance_map[other_station] = distance

        return cls(stations)

    @property
    def num_stations(self):
        return len(self.stations)

    def __repr__(self):
        return f"{self.__class__.__name__}(num_stations={self.num_stations!r})"
    
    def _get_distance(self, path: list[Station]):
        distance = 0
        for i, station in enumerate(path[:-1]):
            next_station = path[i + 1]
            distance += station.distance_map[next_station]
        return distance

    def solve_tsp_slowly(self, start_station: Station):
        best_path: list[Station] = None
        best_distance: float = math.inf
        search_count = 0

        def recurse(curr_path: list[Station], stations_left: set[Station]):
            nonlocal best_path
            nonlocal best_distance
            nonlocal search_count

            if len(stations_left) == 0:
                # update using new path
                search_count += 1
                distance = self._get_distance(curr_path)
                if distance < best_distance:
                    best_path = curr_path
                    best_distance = distance

            for station in stations_left:
                # make a shallow copy and change
                new_path = curr_path + [station]
                new_stations_left = stations_left - set([station])
                recurse(new_path, new_stations_left)
        
        recurse([start_station], set(self.stations) - set([start_station]))
        return best_path, best_distance
    
    def _random_choice_distribution(self, elements, probabilites):
        assert len(elements) == len(probabilites)
        probability_sum = sum(probabilites)
        random_num = random.random() * probability_sum
        for probability, element in zip(probabilites, elements):
            random_num -= probability
            if random_num <= 0:
                return element
        assert 1 == 2, "This should never happen."
    
    def antcolony_optimization(self, start_station: Station, iterations=100, distance_power=2,
                               pheromone_strength=0.1, num_ants=100, pheromone_evaporation_factor=0.9,
                               print_progress=False):
        pheromones = {
            s1: {
                s2: 1. for s2 in self.stations if s2 < s1
            }
            for s1 in self.stations
        }

        def get_pheromone(station1: Station, station2: Station) -> float:
            assert station1 != station2, "Stations must be different"
            if station1 > station2:
                return pheromones[station1][station2]
            else:
                return pheromones[station2][station1]
            
        def set_pheromone(station1: Station, station2: Station, value):
            assert station1 != station2, "Stations must be different"
            if station1 > station2:
                pheromones[station1][station2] = value
            else:
                pheromones[station2][station1] = value

        best_path: list[Station] = []
        best_path_length = math.inf

        for iteration_index in range(iterations):
            if print_progress:
                print(f"iteration {iteration_index + 1}/{iterations}", end="\r")

            ant_paths = []

            # simulate ants
            for ant_index in range(num_ants):
                curr_station = start_station
                path: list[Station] = [curr_station]
                stations_left = set(self.stations) - set(path)

                while len(stations_left) > 0:
                    curr_station = self._random_choice_distribution(stations_left, [
                        (
                            get_pheromone(curr_station, station) * pheromone_strength
                            / (curr_station.distance_map[station] ** distance_power)
                        )
                        for station in stations_left
                    ])

                    stations_left.remove(curr_station)
                    path.append(curr_station)

                ant_paths.append(path)
            
            # update pheromones based on ant paths
            for path in ant_paths:
                path_length = self._get_distance(path)
                pheromone_addition = 1 / (path_length / len(path))
                for i, station in enumerate(path[:-1]):
                    next_station = path[i + 1]
                    pheromone = get_pheromone(station, next_station)
                    set_pheromone(station, next_station, pheromone_addition + pheromone)

                # update best path
                if path_length < best_path_length:
                    best_path_length = path_length
                    best_path = path
                    
                    save_new_best([self.stations.index(s) for s in path], path_length)

            # evaporate all pheromones a bit
            for station1 in self.stations:
                for station2 in self.stations:
                    if not station1 > station2:
                        continue
                    value = get_pheromone(station1, station2)
                    set_pheromone(station1, station2, value * pheromone_evaporation_factor)

        if print_progress:
            print("finished.")

        return best_path, best_path_length
    
    def show(self, path: list[Station], img_size=1000, station_radius=20, line_width=5, save_path=None):
        # Create a {img_size}x{img_size} black image
        image = Image.new("RGB", (img_size, img_size), color="black")
        draw = ImageDraw.Draw(image)

        def scale(coord):
            return int(coord * (img_size - 2 * 20) + 20)

        # Draw the paths 
        for i in range(len(path) - 1):
            s1 = path[i]
            s2 = path[i + 1]
            x1, y1 = scale(s1.position[0]), scale(s1.position[1])
            x2, y2 = scale(s2.position[0]), scale(s2.position[1])
            draw.line([(x1, y1), (x2, y2)], fill="white", width=line_width)

        # Draw each station
        for station in self.stations:
            x = scale(station.position[0])
            y = scale(station.position[1])
            draw.ellipse(
                [(x - station_radius, y - station_radius), (x + station_radius, y + station_radius)],
                fill="blue"
            )

        if save_path is None:
            image.show()
        else:
            image.save(save_path)

    def get_distance_matrix(self):
        return [
            [
                station1.distance_map[station2]
                for station2 in self.stations
            ]
            for station1 in self.stations
        ]

    def to_json_data(self):
        return {
            "node_positions": [
                station.position.tolist()
                for station in self.stations
            ],
            "distance_matrix": self.get_distance_matrix()
        }

    @classmethod
    def from_json_data(cls, data):
        distance_matrix = data["distance_matrix"]
        station_positions = data["node_positions"]

        stations = [Station(pos) for pos in station_positions]
        for i, station1 in enumerate(stations):
            station1.distance_map = {}
            for j, station2 in enumerate(stations):
                station1.distance_map[station2] = distance_matrix[i][j]

        return cls(stations)

In [3]:
# let's try it with linear programming

from scipy.optimize import linprog
import json
from datetime import timedelta

with open("graph-exports/graph2.txt", "r") as file:
    network_data = json.load(file)

network = TramNetwork.from_json_data(network_data)
kinkelstrasse = network.stations[19]

edges = network.get_distance_matrix()

In [9]:
import json
from datetime import timedelta

with open("graph-exports/graph2.txt", "r") as file:
    network_data = json.load(file)

network = TramNetwork.from_json_data(network_data)
kinkelstrasse = network.stations[19]

optimal_path, total_seconds = network.antcolony_optimization(
    start_station=kinkelstrasse,
    iterations=999999,
    num_ants=1000,
    print_progress=True)

total_time = timedelta(seconds=total_seconds)
print(f"total time: {total_time}")

network.show(optimal_path, station_radius=5, line_width=2, save_path="optimal-normal-tsp-bad.png")


KeyboardInterrupt: 

In [4]:
import json
from datetime import timedelta

with open("graph-exports/graph2.txt", "r") as file:
    network_data = json.load(file)

network = TramNetwork.from_json_data(network_data)

with open("exports/000000049-24210.txt", "r") as file:
    optimal_indeces = json.load(file)
    optimal_path = [network.stations[i] for i in optimal_indeces]
    total_seconds = network._get_distance(optimal_path)

total_time = timedelta(seconds=total_seconds)
print(f"total time: {total_time}")

network.show(optimal_path, station_radius=5, line_width=2)


total time: 6:43:30


In [13]:
print([network.stations.index(s) for s in optimal_path])

[19, 18, 17, 16, 15, 133, 36, 37, 38, 39, 40, 41, 28, 0, 12, 13, 14, 185, 186, 187, 188, 198, 199, 200, 201, 202, 189, 190, 191, 192, 82, 83, 84, 85, 86, 87, 88, 89, 44, 43, 4, 5, 6, 7, 8, 9, 10, 11, 3, 2, 42, 1, 132, 131, 130, 35, 34, 33, 32, 26, 27, 31, 30, 129, 61, 62, 97, 96, 95, 94, 93, 92, 91, 90, 47, 48, 49, 180, 63, 114, 115, 116, 117, 118, 119, 120, 121, 46, 45, 25, 81, 80, 79, 50, 78, 69, 68, 67, 66, 65, 64, 51, 60, 59, 58, 57, 56, 55, 54, 53, 52, 72, 73, 167, 168, 74, 76, 75, 179, 176, 177, 178, 181, 182, 183, 184, 23, 22, 21, 20, 24, 140, 141, 142, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 169, 170, 171, 172, 173, 174, 175, 143, 144, 145, 146, 147, 148, 149, 150, 151, 197, 196, 195, 194, 193, 77, 70, 71, 134, 135, 136, 137, 138, 139, 29, 128, 127, 126, 125, 162, 163, 164, 165, 166, 124, 123, 122, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152]
